<a href="https://colab.research.google.com/github/h1-ti/GPEN/blob/main/GPEN_FACE_ENHANCER_VIA_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **実行前の準備**



1.   GOOGOLE DRIVE を開き左上の『＋新規』→　フォルダ　→　faceset フォルダを作成
2.   ローカル（自分のパソコン）の 1)Faceset/CATEGORYフォルダ を開き、超解像したいSRCをzipで圧縮する。例）1)Faceset/hogezaka/hoge -> 1)Faceset/hogezaka/hoge.zip
3.1のfacesetフォルダに2のhoge.zipをアップロード（複数可）



# **実行**



* このページの上にある　ランタイム（左から５番目）　→　すべてのセルを実行

## ※注意  **1.MOUNT GOOGLE DRIVE** のところで入力があります

### 詳しく

* GOOGLE DRIVEのマウントが完了したら終わるまで待つだけです


* 完了すると先ほどのfacesetフォルダにhoge_superres.zipができるのでドライブからダウンロードしてください


* ※srcが多すぎると最後まで完了せずに接続が切れるかもしれません。時間をおいてやり直してください。完了したものはスキップされるようになっております

* 実行中に新しいsrcのzipをアップすることも可能です


* ※また、2.INSTALL and GET MODEL WEIGHTでダウンロードがうまくいかず止まっている場合はやり直してください

**CHECK VIDEO CARD'S SPEC**


速さ比較

 P100(?f/s) > T4(6f/s) > K80(3f/s)

In [ ]:
!nvidia-smi

# **1.MOUNT GOOGLE DRIVE**

### ①Go to this URL in a browser:の右のリンクからGOOGLEにログイン
### ②Enter your authorization code:の下に①で取得したコードをペーストしてGOOGLE DRIVE をマウント



In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# **2.INSTALL and GET MODEL WEIGHT**

### install GPEN

In [ ]:
%pip install ninja

%cd /content
!git clone https://github.com/h1-ti/GPEN.git

### get weights

In [ ]:
%cd /content/GPEN/weights
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/RetinaFace-R50.pth
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-BFR-512.pth

# **3.FACE ENHANCE PROCESS**


In [ ]:
%cd /content/GPEN

/content/GPEN


In [ ]:
#@title FACE ENHANCE PROCESS

from face_enhancement import FaceEnhancement
from tqdm import tqdm
import os, os.path as osp
import zipfile
import glob
import cv2
import shutil
import numpy as np

from DFLIMG.DFLPNG import DFLPNG
from DFLIMG.DFLJPG import DFLJPG

root = "/content/GPEN"

model = {'name':'GPEN-BFR-512', 'size':512}
    
faceset_dir = "/content/drive/MyDrive/faceset"
indir = osp.join(root, "temp_input")
os.makedirs(indir, exist_ok=True)

faceenhancer = FaceEnhancement(base_dir=root, size=model['size'], model=model['name'], channel_multiplier=2)

done = []

while True:
    src_names = [osp.splitext(d)[0] for d in os.listdir(faceset_dir)
             if osp.splitext(d)[1] == ".zip" and "_superres" not in d
             and not osp.exists(osp.join(faceset_dir, osp.splitext(d)[0]+"_superres.zip"))
             and osp.splitext(d)[0] not in done]
    if len(src_names) == 0:
        break
    print(src_names)
    


    for id, src in enumerate(src_names):
        print("{} / {} [{}]".format(id+1, len(src_names), src))
        zip_path = osp.join(faceset_dir, src+".zip")
        print("Extracting [{}] ...".format(zip_path))
        with zipfile.ZipFile(zip_path) as z:
            z.extractall(indir)

        files = sorted(glob.glob(osp.join(indir, src, '*.*g')))

        outdir = osp.join(root, src+"_superres", src+"_superres")
        os.makedirs(outdir, exist_ok=True)

        for n, file in enumerate(tqdm(files[:], total=len(files), desc="Enhancing faces ...")):
            filename = osp.basename(file)
            ext = osp.splitext(file)[1]
            if ext == ".jpg":
                dflimg = DFLJPG.load(file)
            elif ext == ".png":
                dflimg = DFLPNG.load(file)
            else:
                continue

            if not dflimg:
                continue
            
            im = cv2.imread(file, cv2.IMREAD_COLOR) # BGR
            input_x, input_y = im.shape[:2]
            if not isinstance(im, np.ndarray): print(filename, 'error'); continue
            im = cv2.resize(im, (0,0), fx=2, fy=2)

            img, orig_faces, enhanced_faces = faceenhancer.process(im)
            img = cv2.resize(img, (input_x, input_y))
            cv2.imwrite(osp.join(outdir, filename), img)

            if ext == ".jpg":
                _fanseg_mask = dflimg.dfl_dict.get('fanseg_mask', None)
                if _fanseg_mask is not None:
                    ret, buf = cv2.imencode( '.jpg', _fanseg_mask, [int(cv2.IMWRITE_JPEG_QUALITY), 85] )

                    if ret and len(buf) < 64000:
                        _fanseg_mask = buf
                    else:
                        io.log_err("Unable to encode fanseg_mask for %s" % (filename) )
                        _fanseg_mask = None

                _xseg_mask = dflimg.dfl_dict.get('xseg_mask', None)
                if _xseg_mask is not None:
                    ret, buf = cv2.imencode( '.jpg', _xseg_mask, [int(cv2.IMWRITE_JPEG_QUALITY), 85] )

                    if ret and len(buf) < 64000:
                        _xseg_mask = buf
                    else:
                        io.log_err("Unable to encode xseg_mask for %s" % (filename) )
                        _xseg_mask = None

                DFLJPG.embed_dfldict (osp.join(outdir, filename), 
                                        {'face_type': dflimg.get_face_type(),
                                            'landmarks': dflimg.get_landmarks(),
                                            'ie_polys' : dflimg.get_ie_polys(),
                                            'source_filename': dflimg.get_source_filename(),
                                            'source_rect': dflimg.get_source_rect(),
                                            'source_landmarks': dflimg.get_source_landmarks(),
                                            'image_to_face_mat': dflimg.get_image_to_face_mat(),
                                            'fanseg_mask' : _fanseg_mask,
                                            'xseg_mask' : _xseg_mask,
                                            'eyebrows_expand_mod' : None,
                                            'relighted' : None,
                                            "histgram" : None,
                                            "recognition" : dflimg.get_recognition(),
                                        })
            elif ext == ".png":
                dflimg = DFLPNG.embed_dfldict (osp.join(outdir, filename), 
                                        {'face_type': dflimg.get_face_type(),
                                            'landmarks': dflimg.get_landmarks(),
                                            'ie_polys' : dflimg.get_ie_polys(),
                                            'source_filename': dflimg.get_source_filename(),
                                            'source_rect': dflimg.get_source_rect(),
                                            'source_landmarks': dflimg.get_source_landmarks(),
                                            'image_to_face_mat': dflimg.get_image_to_face_mat(),
                                            'fanseg_mask' : dflimg.dfl_dict.get ('fanseg_mask', None),
                                            'xseg_mask' : dflimg.dfl_dict.get('xseg_mask', None),
                                            'eyebrows_expand_mod' : None,
                                            'relighted' : None,
                                            "histgram" : None,
                                            "recognition" : dflimg.get_recognition(),
                                        })


        
        shutil.make_archive(src+"_superres", 'zip', src+"_superres")
        if osp.exists(osp.join(faceset_dir, src+"_superres.zip")):
            os.remove(osp.join(faceset_dir, src+"_superres.zip"))
        shutil.move(src+"_superres.zip", faceset_dir)
        shutil.rmtree(src+"_superres")
        print("Successfully completed {}".format(src+"_superres"))
        done.append(src)

print(done)